# Strategy 01 Periodic Performance Analysis
This notebook explores the time-based summary workbook generated by `result_analysis.py`.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from glob import glob

In [ ]:
sns.set_theme(style='whitegrid', context='talk')

## Locate Periodic Workbooks

In [ ]:
downloads_dir = Path(r'c:/Users/by003457/downloads')
period_suffix = 'company_revenue_period_analysis_'
excel_pattern = f'{period_suffix}*.xlsx'
periodic_files = sorted(downloads_dir.glob(excel_pattern))
periodic_files

Choose the workbook you want to analyse by setting `analysis_file`.

In [ ]:
if not periodic_files:
    raise FileNotFoundError('No periodic analysis workbooks were found in the downloads directory.')
analysis_file = periodic_files[-1]  # use most recent by default
analysis_file

In [ ]:
sheets = pd.read_excel(analysis_file, sheet_name=None)
list(sheets.keys())

In [ ]:
monthly_df = sheets.get('monthly_overview', pd.DataFrame()).copy()
annual_df = sheets.get('annual_overview', pd.DataFrame()).copy()
ticker_yearly_df = sheets.get('ticker_yearly', pd.DataFrame()).copy()
monthly_df.head(2)

## Monthly Trends

In [ ]:
if monthly_df.empty:
    raise ValueError('monthly_overview sheet is empty. Ensure the workbook was generated with recent runs.')
monthly_df['period'] = pd.PeriodIndex(monthly_df['sell_year_month'], freq='M').to_timestamp()
monthly_df.sort_values('period', inplace=True)
monthly_df[['period', 'total_revenue', 'trade_count', 'avg_profit_pct']].tail()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
sns.lineplot(data=monthly_df, x='period', y='total_revenue', marker='o', ax=ax)
sns.scatterplot(data=monthly_df, x='period', y='total_revenue', hue='win_ratio', palette='viridis', ax=ax)
ax.set_title('Monthly Total Revenue with Win Ratio shading')
ax.set_xlabel('Month')
ax.set_ylabel('Total Revenue')
ax.grid(True)
fig.autofmt_xdate()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
sns.lineplot(data=monthly_df, x='period', y='avg_profit_pct', marker='o', ax=ax, color='darkorange')
ax.set_title('Monthly Average Profit %')
ax.set_xlabel('Month')
ax.set_ylabel('Average Profit %')
ax.axhline(0, color='black', linewidth=1)
ax.grid(True)
fig.autofmt_xdate()

## Annual Performance

In [ ]:
if annual_df.empty:
    print('Annual overview sheet is empty; skipping annual analysis.')
else:
    annual_df.sort_values('sell_year', inplace=True)
    display(annual_df)
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(data=annual_df, x='sell_year', y='total_revenue', hue='win_ratio', dodge=False, palette='Blues', ax=ax)
    ax.set_title('Annual Total Revenue (Win Ratio Shaded)')
    ax.set_xlabel('Year')
    ax.set_ylabel('Total Revenue')
    ax.legend(title='Win Ratio')
    plt.show()

## Year-on-Year Momentum

In [ ]:
if not annual_df.empty:
    yoy_cols = ['sell_year', 'total_revenue', 'revenue_yoy_pct', 'avg_profit_pct', 'win_ratio']
    display(annual_df[yoy_cols])
else:
    print('YoY metrics unavailable; annual overview missing.')

## Ticker-Level Trends

In [ ]:
if ticker_yearly_df.empty:
    print('Ticker yearly sheet missing; skipping ticker-level analysis.')
else:
    ticker_yearly_df.sort_values(['tickerSymbol', 'sell_year'], inplace=True)
    top_tickers = ticker_yearly_df.groupby('tickerSymbol')['total_revenue'].sum().nlargest(5).index.tolist()
    ticker_yearly_df.query('tickerSymbol in @top_tickers', inplace=True)
    ticker_yearly_df.head(10)

In [ ]:
if not ticker_yearly_df.empty:
    fig, ax = plt.subplots(figsize=(14, 6))
    sns.lineplot(data=ticker_yearly_df, x='sell_year', y='total_revenue', hue='tickerSymbol', marker='o', ax=ax)
    ax.set_title('Top Tickers: Yearly Revenue Trends')
    ax.set_xlabel('Year')
    ax.set_ylabel('Total Revenue')
    ax.grid(True)
    plt.show()

In [ ]:
if not ticker_yearly_df.empty:
    fig, ax = plt.subplots(figsize=(14, 6))
    sns.lineplot(data=ticker_yearly_df, x='sell_year', y='avg_profit_pct', hue='tickerSymbol', marker='o', ax=ax)
    ax.set_title('Top Tickers: Yearly Average Profit %')
    ax.set_xlabel('Year')
    ax.set_ylabel('Average Profit %')
    ax.axhline(0, color='black', linewidth=1)
    ax.grid(True)
    plt.show()

## Rolling Metrics Example

In [ ]:
monthly_df = monthly_df.set_index('period')
monthly_df['rolling_revenue_3m'] = monthly_df['total_revenue'].rolling(window=3).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
monthly_df[['total_revenue', 'rolling_revenue_3m']].plot(ax=ax)
ax.set_title('3-Month Rolling Revenue')
ax.set_xlabel('Month')
ax.set_ylabel('Revenue')
ax.grid(True)
plt.show()

## Save Processed Data

In [ ]:
output_dir = analysis_file.parent / 'analysis_outputs'
output_dir.mkdir(parents=True, exist_ok=True)
monthly_df.reset_index().to_csv(output_dir / 'monthly_overview_processed.csv', index=False)
annual_df.to_csv(output_dir / 'annual_overview_processed.csv', index=False)
ticker_yearly_df.to_csv(output_dir / 'ticker_yearly_processed.csv', index=False)

Notebook finished. Adjust the plots or aggregations above to explore additional time-series insights.